# **Model Creation**

## **Importing Libraries**

In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

## **Transformation on Dataset**

In [6]:
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor()
])

## **Loading Fashion MNIST Dataset**

In [7]:
train_dataset = datasets.FashionMNIST(root="/data", train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root="/data", train=False, download=True, transform=transform)

## **Train and Validation Dataset Length**

In [8]:
print(len(train_dataset))
print(len(test_dataset))

60000
10000


## **Classes in Dataset**

In [9]:
print(train_dataset.classes)

['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


## **Creating CNN Model**

In [10]:
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),

    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),

    nn.Flatten(),
    nn.Dropout(0.4),
    nn.Linear(64*7*7, 10)
)

## **Creating DataLoader**

In [12]:
train_loader = DataLoader(train_dataset, batch_size=200, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=True)

## **Loss and Optimizer**

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, weight_decay=0.001)

## **Training the Model**

In [15]:
model.train()

for epoch in range(11):
  total_loss = 0.0
  for images, labels in train_loader:
    predicted_labels = model(images)
    loss = criterion(predicted_labels, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  epoch_loss = total_loss / len(train_loader)
  print(f'Epoch: {epoch}, Loss: {epoch_loss:.4f}')

Epoch: 0, Loss: 0.6090
Epoch: 1, Loss: 0.5455
Epoch: 2, Loss: 0.5072
Epoch: 3, Loss: 0.4811
Epoch: 4, Loss: 0.4622
Epoch: 5, Loss: 0.4465
Epoch: 6, Loss: 0.4322
Epoch: 7, Loss: 0.4237
Epoch: 8, Loss: 0.4123
Epoch: 9, Loss: 0.4038
Epoch: 10, Loss: 0.3968


## **Calculating Accuracy on Validation Dataset**

In [17]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        output_label = model(images)
        _, predicted = torch.max(output_label, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    accuracy = correct / total
    print(f'Accuracy on Validation Dataset: {accuracy * 100:.2f}%')

Accuracy on Validation Dataset: 86.43%


# **Testing Model and Creating Web App**

## **Defining Class Labels**

In [59]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## **Defining Prediction Function**

In [60]:
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])


def predict(image):
    image = transform(image).unsqueeze(0)
    model.eval()
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    return class_names[predicted.item()]

## **Creating Gradio Interface**

In [62]:
import gradio as gr

interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(
        type="pil",
        label="Upload a Clothing Image",
        image_mode="RGB"
    ),
    outputs=gr.Label(label="Predicted Clothing Type"),
    title="👕 Fashion MNIST Clothing Classifier",
    description="Upload any image of clothing. The model will classify it as one of 10 fashion items.",
    theme=gr.themes.Soft()
)

interface.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2e3db92d23d8a48ca7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d650900511741b39c4.gradio.live
Killing tunnel 127.0.0.1:7860 <> https://2e3db92d23d8a48ca7.gradio.live
